In [87]:
import requests, re, json
from os import listdir
from time import sleep
from bs4 import BeautifulSoup as soup

In [81]:
raw_cookies = ""
cookies = dict()
for cookie in raw_cookies.split(';'):
    key, val = cookie.split('=')
    cookies[key.strip()] = val.strip()

In [82]:
def get_articles(page:int=1, cookies:dict={}):
    url = "https://arca.live/b/cyoa?&category=%EC%9E%90%EC%9E%91&p="+str(page)
    res = requests.get(url, cookies=cookies)
    
    a = soup(res.text).find_all('a', {'class':'vrow column'})
    return ['https://arca.live'+i['href'] for i in a]

In [88]:
class article():
    def __init__(self, url:str):
        self.url = url
        self.number = self.url.split('/')[-1].split('?')[0]
    
    def error(self, text):
        print(f'Pass {self.number} - {text}')
        
        
    def get(self, cookies:dict={}, save_path:str='./', delay:float=0):
        
        # Pass existing file
        if(self.number+'.json' in listdir(save_path)):
            self.error('Already exists')
            return
        
        # I don't want to be banned from the site
        sleep(delay)
        
        try:
            self.res = requests.get(self.url, cookies=cookies)
            self.soup = soup(self.res.text)
        except:
            self.error('Check internet connection')
            return
        
        # Article information
        try:
            self.author = self.soup.find("meta", {"name":"author"}).attrs['content']
            self.title = self.soup.find("meta", {"name":"title"}).attrs['content']
            self.recommand = int(
                self.soup.find(string='추천').next.next.contents[0]
                )
        except:
            self.error('Failed to get article information. Banned?')
            return
        
        # Main contents
        self.raw_content = self.soup.find('div', {'class':'article-content'}).getText("\n")
        self.clear_content = re.sub(r'http\S+', '', self.raw_content)
        
        # Save it
        data = {'number':self.number,
                'author':self.author,
                'title':self.title,
                'recommand':self.recommand,
                'clear_content':self.clear_content}
        
        try:
            with open(save_path+self.number+'.json', 'w', encoding='utf8') as f:
                json.dump(data, f, ensure_ascii=True, indent=4)
            
            print(f'Done {self.number} - {self.title}')
        except:
            self.error('Failed to save data. Try with administration.')

In [ ]:
from random import random
pages = 50

for i in range(1, pages+1):
    for page in get_articles(i, cookies):
        article(page).get(cookies=cookies, save_path='data/', delay=random()*2)